In [1]:
from pymongo import MongoClient

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import time

import re
from urllib.parse import urlparse

from bs4 import BeautifulSoup

from IPython.display import clear_output


In [2]:

client = MongoClient('mongodb://localhost:27017/')
db = client['Lowerated']
collection = db['RottenTomatoes']

CollectionList = list(collection.find({}))
CollectionList[0]


{'_id': 0,
 'url': 'https://www.rottentomatoes.com/m/shawshank_redemption',
 'audienceScore': '98%',
 'criticsScore': '89%',
 'Critic Reviews': [{'ReviewID': 1,
   'Critic': "Buffa’s Buffet\nSubscribe\nSign in\nDiscover more from Buffa’s Buffet\nBaseball and movie commentary, deep dive TV show discussion, random hockey and assorted sports takes, dad humor, and basically unfiltered access to my filthy mind.\nSubscribe\nContinue reading\nSign in\nWhy ‘Shawshank Redemption’ sits atop the rewatchable movies list\nFrank Darabont’s prison opus plays like a hit song, 30 years later.\nDAN BUFFA\nAPR 06, 2024\n2\nShare\nThere’s something unique about a great movie that also happens to be highly rewatchable. It’s not as common as a nice picture or a film that was well made yet doesn’t sit atop the pile of discs or streamer options for a revisit. Some sets are best seen once, before being broken down and discarded in your head. So if Schindler’s List is at the top of the “hard to rewatch” stack, 

In [3]:

driver = webdriver.Chrome()
driver.maximize_window()


In [4]:

for data in CollectionList:
    
    movie_id = data['_id']
    if movie_id > 34:
        url = data['url']
        driver.get(url)
        time.sleep(1)

        driver.execute_script("window.scrollBy(0, 1100);")

        try:
            TotalAudience = driver.find_element(By.XPATH, '//*[@id="modules-wrap"]/section[2]/div[1]/rt-link')
        except Exception as e:
            print('No Audiance Review found')
            TotalAudience = None
                        
                        
        if TotalAudience:
            Totaltxt = TotalAudience.text
            
            match = re.search(r'\((\d+)\+', Totaltxt)
            if match:
                TotalAudienceReviews = int( match.group(1) )  
            else:
                
                match = re.search(r'\((\d+)\)', Totaltxt)
                if match:
                    TotalAudienceReviews = int(match.group(1))  
                else:
                    TotalAudienceReviews = None
                    print('Number not found Regex')    
        
        
        
            if TotalAudienceReviews:
                parsed_url = urlparse(url)
                movie_name = parsed_url.path.split('/m/')[-1]
            
                driver.get(f'https://www.rottentomatoes.com/m/{movie_name}/reviews?type=user')
                time.sleep(1)

            CurrentMovieAllReviews = []
            TotalAudienceReviews -= 1
            for ReviewNum in range(1, TotalAudienceReviews):
                
                # --------------------- Press Load More after every 20 Reviews ---------------------
                if ReviewNum % 20 == 0:
                    print(f"Now more reviews are available and uploaded Total ReviewNum completed are: {ReviewNum}", end= " ")
                    MoreReviews = driver.find_element(By.XPATH, '//*[@id="reviews"]/div[2]/rt-button')
                    MoreReviews.click()                         
                
                        
                try:                                                                                                    
                    ReviewElement = driver.find_element(By.XPATH, f'//*[@id="reviews"]/div[1]/div[{ReviewNum}]/div[2]/drawer-more/p')
                    Reviewtxt = ReviewElement.text
                    
                    print(Reviewtxt[:10])
                    
                except NoSuchElementException:
                    print('EXCEPTION No Element found')
                    time.sleep(360)

                driver.execute_script("window.scrollBy(0, 150);")
                time.sleep(1)
                        
                CurrentReview ={
                    'ReviewID':	ReviewNum,
                    'Critic': Reviewtxt,
                }
                        
                CurrentMovieAllReviews.append(CurrentReview)
                
            movie_id = data['_id']
            CurrentMovie = {
                    "_id": movie_id,
                    "Reviews": CurrentMovieAllReviews
                }
                    
            print(f'Total reviews: {ReviewNum} collected for movie: {movie_name}, so Total movies done: {movie_id}')    
                    
            collection.update_one(
                {'_id': CurrentMovie['_id']},
                {'$set': {'Audience Reviews': CurrentMovie['Reviews']}},
                upsert=True
            )
            
            clear_output(wait=True)
        

It probabl
Grand scor
Amazing mo
A true cla
Better the
What a gre
The film i
So so amaz
The best D
An instant
Lion King 
Best Disne
O filme é 
This film 
I don’t un
I don't th
I don't th
The Lion K
Melhor fil
Now more reviews are available and uploaded Total ReviewNum completed are: 20 Shakespear
EXCEPTION No Element found
This reall
One of the
Yes. An al
Back befor
Classic mo
Though not
Instantly 
There is o
Loved the 
Its really
Lion King 
Amazing mo
A complete
This anima
I really l
I love The
My favorit
You just c
Now more reviews are available and uploaded Total ReviewNum completed are: 40 Back when 
EXCEPTION No Element found
Whether yo
My favorit
While the 
Just so pe
My God, th
This movie
The movie 
This is on
A beloved 
The Lion K
A gripping
such a goo
In front o
The best D
☆It's The 
Disneys be
One of the
This is TH
Now more reviews are available and uploaded Total ReviewNum completed are: 60 This is a 
EXCEPTION No Element found
Great! Cla
Las cancio
Best movie
it's a won
Let

KeyboardInterrupt: 

In [1]:
ReviewNum, movie_id

NameError: name 'ReviewNum' is not defined

: 

In [27]:

CurrentMovie = {
    "_id": movie_id,
    "Reviews": CurrentMovieAllReviews
}

collection.update_one(
    {'_id': CurrentMovie['_id']},
    {'$set': {'Audience Reviews': CurrentMovie['Reviews']}},
    upsert=True
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)